In [1]:
# Importing programs, porting in database
import sqlite3
import pandas as pd
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [2]:
# Creating dataset for each team with total number of home
# games played, and home goals scored.
c.execute("""select distinct HomeTeam as team,
                    count(HomeTeam) as homeGames,
                    sum(FTHG) as homeGoals
                    from matches
                    where Season == 2011
                    group by 1;""")
df_home = pd.DataFrame(c.fetchall())
df_home.columns = [x[0] for x in c.description]

In [3]:
# Creating dataset for each team with total number of away
# games played, and away goals scored.
c.execute("""select distinct AwayTeam as team,
                    count(AwayTeam) as awayGames,
                    sum(FTAG) as awayGoals
                    from matches
                    where Season == 2011
                    group by 1;""")
df_away = pd.DataFrame(c.fetchall())
df_away.columns = [x[0] for x in c.description]

In [4]:
# Merging datasets
df_total = pd.merge(df_home, df_away, on="team")

In [5]:
# Summing total games played.
df_total['totalGamesplayed'] = df_total.apply(
    (lambda x: x['homeGames']+x['awayGames']), axis=1)

In [6]:
# Summing total goals earned.
df_total['totalPointsearned'] = df_total.apply(
    (lambda x: x['homeGoals']+x['awayGoals']), axis=1)

In [7]:
# Dropping unnecessary columns
df_total = df_total.drop(
    columns=['homeGames', 'homeGoals', 'awayGames', 'awayGoals'])

In [8]:
# Creating list of home teams, away teams, and winners
c.execute("""select date, HomeTeam, AwayTeam, FTR
                    from matches
                    where Season == 2011;""")
df_res = pd.DataFrame(c.fetchall())
df_res.columns = [x[0] for x in c.description]

In [9]:
# Changing date to datetime
df_res['Date'] = df_res.Date.map(lambda x:
                                 pd.to_datetime(x[:10],
                                                format='%Y/%m/%d'))

In [10]:
# Creating a list of winning and losing teams for each game
df_res['gamesWon'] = df_res.apply((lambda x: x['HomeTeam'] if x['FTR'] == "H" else (
    x['AwayTeam'] if x['FTR'] == "A" else "Draw")), axis=1)

df_res['gamesLost'] = df_res.apply((lambda x: x['HomeTeam'] if x['FTR'] == "A" else (
    x['AwayTeam'] if x['FTR'] == "H" else "Draw")), axis=1)

In [11]:
# Counting number of games each team has won and lost
lc_df = df_res['gamesLost'].value_counts()
wc_df = df_res['gamesWon'].value_counts()

In [12]:
# Resetting indexes
lc_df = lc_df.reset_index()
wc_df = wc_df.reset_index()

In [13]:
# Merge win and loss datasets
df_all = pd.merge(wc_df, lc_df, on="index")

In [14]:
# Change column name
df_all = df_all.rename(columns={'index': 'team'})

In [15]:
# Merge datasets
df_total = df_total.merge(df_all, how="left", on="team")

In [17]:
df_total.head()

,team,totalGamesplayed,totalPointsearned,gamesWon,gamesLost
0,Aachen,34,30,6,15
1,Arsenal,38,74,21,10
2,Aston Villa,38,37,7,14
3,Augsburg,34,36,8,12
4,Bayern Munich,34,77,23,7


----

## Obtain subset of matches that had rain

In [51]:
import weather_detector

In [52]:
#Insert Dark Sky API key below.      See: https://darksky.net/dev/account
api_key = "" 

In [47]:
rain_table = weather_detector.rain_table("2011", api_key)

In [50]:
rain_table.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,precipType
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,rain
1,1189,D1,2011,2012-03-31,Augsburg,FC Koln,2,1,H,rain
2,1206,D1,2011,2012-03-31,Leverkusen,Freiburg,0,2,A,rain
3,1215,D1,2011,2012-03-31,Kaiserslautern,Hamburg,0,1,A,rain
4,1396,D2,2011,2012-03-31,Munich 1860,Hansa Rostock,0,1,A,rain


## Something or another

In [23]:
teams = list(df_total['team'])

In [25]:
league = []
for team in teams:
    team_entry = dict({"name": team, "goals": 45, "wins": 14, "losses": 20, "hist": str("hlelo")})
    league.append(team_entry)

## Load into MongoDB

In [27]:
import pymongo
client = pymongo.MongoClient("mongodb://127.0.0.1:27017")
db = client['module_2_assessment']
coll = db['german_teams']
for team in range(0,len(league)):
    coll.insert_one(league[team])

DuplicateKeyError: E11000 duplicate key error collection: module_2_assessment.german_teams index: _id_ dup key: { : ObjectId('5d4de6b74164e49b412ac79f') }

In [ ]:
coll.find_one({})